# XGBoost

## Import

In [6]:
# Preprocessing
import pandas as pd
import numpy as np
from dask import dataframe as dd 
from sklearn.model_selection import train_test_split

# Import de modèle
import dask_xgboost
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Connection cluster
from sklearn.externals import joblib
from dask.distributed import Client
try:
    client1.close()
except:
    print(' pas de client')
#client1 = Client('c128-26.enst.fr:8786')  # set up local cluster on your laptop
client1 = Client()
client1

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:49314 remote=tcp://127.0.0.1:49263>


Client Scheduler: tcp://127.0.0.1:49387 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.36 GB


## Calcul du modèle

In [2]:
xtrain = pd.read_csv('xtrain_challenge.csv')
ytrain = pd.read_csv('ytrain_challenge.csv')
dataset=xtrain.merge(ytrain,  left_index=True,right_index=True)
train,test=train_test_split(dataset,test_size=0.1,stratify=dataset['y'])

In [8]:
bst = xgb.XGBClassifier(max_depth=100,n_estimators=200,n_jobs=-1)

paramGrid_lat = {
        'gamma':[0.0001, 0.001,0.01],
        'learning_rate':[0.3,0.4,0.6,0.7]
    }

gsc=GridSearchCV(bst,param_grid=paramGrid_lat,iid=False,cv=2,n_jobs=-1)

In [9]:
#with joblib.parallel_backend("dask",scatter=[train.drop('y',axis=1),train.y]):
#    print('start')
gsc.fit(train.drop('y',axis=1),train.y)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

## Sauvegarde des résultats et affichage

In [ ]:
result=pd.DataFrame(gsc.cv_results_).sort_values('rank_test_score')
result.to_csv('XGB.csv',index=None)
result

In [ ]:
filename = 'XGB_grid.sav'
joblib.dump( gsc, filename)

In [ ]:
yvalid = gsc.predict(test.drop('y',axis=1))
(yvalid == test['y']).mean()

## Calcul du modèle final

In [ ]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=2,n_jobs=11,n_estimators=200,max_depth=100,
                              learning_rate=0.4,gamma=0.0001)
xgb_model.fit(train.drop('y',axis=1), train['y'],
              early_stopping_rounds=7, 
              eval_set=[(test.drop('y',axis=1),  test['y'])])

In [ ]:
yvalid = xgb_model.predict(test.drop('y',axis=1))
(yvalid == test['y']).mean()

In [ ]:
filename2 = 'XGB_final.sav'
joblib.dump( xgb_model, filename2)

## Prepare a file for submission

In [ ]:
# Load test data
xtest = pd.read_csv('xtest_challenge.csv')
model = joblib.load(filename2)
ytest = model.predict(xtest)
print(ytest.shape)
np.savetxt('ytest_xgb_final.csv', ytest, fmt = '%1.0d', delimiter=',')